In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('AnalisisTransacciones').getOrCreate()
sc = spark.sparkContext


In [2]:
rdd = sc.textFile('transacciones.csv')

rdd.take(5)

['id,usuario,monto,fecha,categoria ',
 '1,Juan,1500,2025-03-10,Supermercado',
 '2,Ana,2000,2025-03-12,Tecnología',
 '3,Carlos,500,2025-03-12,Supermercado',
 '4,Juan,3000,2025-03-13,Tecnología']

In [3]:
header = rdd.first()
rdd = rdd.filter(lambda x: x != header)

rdd.take(5)

['1,Juan,1500,2025-03-10,Supermercado',
 '2,Ana,2000,2025-03-12,Tecnología',
 '3,Carlos,500,2025-03-12,Supermercado',
 '4,Juan,3000,2025-03-13,Tecnología',
 '5,Ana,700,2025-03-14,Restaurante']

In [4]:
rdd_mapeado = rdd.map(lambda x: x.split(',')) \
                 .map(lambda x: (x[1], float(x[2]), x[4] ))

rdd_mapeado.collect()


[('Juan', 1500.0, 'Supermercado'),
 ('Ana', 2000.0, 'Tecnología'),
 ('Carlos', 500.0, 'Supermercado'),
 ('Juan', 3000.0, 'Tecnología'),
 ('Ana', 700.0, 'Restaurante')]

In [5]:
rdd_filtrado = rdd_mapeado.filter(lambda x: x[1] > 1000 )

rdd_filtrado.collect()

[('Juan', 1500.0, 'Supermercado'),
 ('Ana', 2000.0, 'Tecnología'),
 ('Juan', 3000.0, 'Tecnología')]

In [6]:
rdd_gastos = rdd_mapeado.map(lambda x: (x[0], x[1])) \
                        .reduceByKey(lambda x,y: x+y)
rdd_gastos.collect()

[('Juan', 4500.0), ('Ana', 2700.0), ('Carlos', 500.0)]

In [7]:
montos = rdd_mapeado.map(lambda x: x[1])

print(montos.collect())

total = montos.sum()
promedio = montos.mean()
maximo = montos.max()
minimo = montos.min()
desviacion = montos.stdev()

print(f'Total: {total}')
print(f'Promedio: {promedio}')
print(f'Maximo: {maximo}')
print(f'Minimo: {minimo}')
print(f'Desviacion: {desviacion}')

[1500.0, 2000.0, 500.0, 3000.0, 700.0]
Total: 7700.0
Promedio: 1540.0
Maximo: 3000.0
Minimo: 500.0
Desviacion: 909.0654541890809


In [8]:
top_usuarios = rdd_gastos.sortBy(lambda x: x[1], ascending=False)

top_usuarios.take(3)


[('Juan', 4500.0), ('Ana', 2700.0), ('Carlos', 500.0)]